# Reshaping and pivot tables
## Reshaping by pivoting DataFrame objects

![reshaping_pivot](img/reshaping_pivot.png)

Data is often stored in so-called “stacked” or “record” format:

```
In [1]: df
Out[1]: 
         date variable     value
0  2000-01-03        A  0.469112
1  2000-01-04        A -0.282863
2  2000-01-05        A -1.509059
3  2000-01-03        B -1.135632
4  2000-01-04        B  1.212112
5  2000-01-05        B -0.173215
6  2000-01-03        C  0.119209
7  2000-01-04        C -1.044236
8  2000-01-05        C -0.861849
9  2000-01-03        D -2.104569
10 2000-01-04        D -0.494929
11 2000-01-05        D  1.071804
```

For the curious here is how the above DataFrame was created:

In [4]:
import numpy as np
import pandas as pd

In [6]:
import pandas._testing as tm

def unpivot(frame):
    N, K = frame.shape
    data = {
        "value": frame.to_numpy().ravel("F"),
        "variable": np.asarray(frame.columns).repeat(N),
        "date": np.tile(np.asarray(frame.index), K),
    }
    return pd.DataFrame(data, columns=["date", "variable", "value"])


df = unpivot(tm.makeTimeDataFrame(3))
df

,date,variable,value
0,2000-01-03,A,0.972916
1,2000-01-04,A,0.168874
2,2000-01-05,A,0.629708
3,2000-01-03,B,-0.350250
4,2000-01-04,B,0.407657
5,2000-01-05,B,-0.935258
6,2000-01-03,C,0.920348
7,2000-01-04,C,1.421589
8,2000-01-05,C,-1.603263
9,2000-01-03,D,1.743644


---

To select out everything for variable `A` we could do:

In [7]:
df[df["variable"] == "A"]

,date,variable,value
0,2000-01-03,A,0.972916
1,2000-01-04,A,0.168874
2,2000-01-05,A,0.629708


But suppose we wish to do time series operations with the variables. A better representation would be where the `columns` are the unique variables and an `index` of dates identifies individual observations. To reshape the data into this form, we use the `DataFrame.pivot()` method (also implemented as a top level function `pivot()`):

In [8]:
df.pivot(index="date", columns="variable", values="value")

variable,A,B,C,D
date,,,,
2000-01-03,0.972916,-0.350250,0.920348,1.743644
2000-01-04,0.168874,0.407657,1.421589,-0.150937
2000-01-05,0.629708,-0.935258,-1.603263,0.294872


If the `values` argument is omitted, and the input `DataFrame` has more than one column of values which are not used as column or index inputs to `pivot`, then the resulting “pivoted” `DataFrame` will have `hierarchical columns` whose topmost level indicates the respective value column:

In [9]:
df["value2"] = df["value"] * 2
pivoted = df.pivot(index="date", columns="variable")
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03  0.972916 -0.350250  0.920348  1.743644  1.945832 -0.700500   
2000-01-04  0.168874  0.407657  1.421589 -0.150937  0.337747  0.815315   
2000-01-05  0.629708 -0.935258 -1.603263  0.294872  1.259416 -1.870516   

                                
variable           C         D  
date                            
2000-01-03  1.840696  3.487288  
2000-01-04  2.843177 -0.301873  
2000-01-05 -3.206525  0.589743

You can then select subsets from the pivoted DataFrame:

In [10]:
pivoted["value2"]

variable,A,B,C,D
date,,,,
2000-01-03,1.945832,-0.700500,1.840696,3.487288
2000-01-04,0.337747,0.815315,2.843177,-0.301873
2000-01-05,1.259416,-1.870516,-3.206525,0.589743


Note that this returns a view on the underlying data in the case where the data are homogeneously-typed.

>Note
>
>`pivot()` will error with a `ValueError: Index contains duplicate entries, cannot reshape` if the index/column pair is not unique. In this case, consider using pivot_table() which is a generalization of pivot that can handle duplicate values for one index/column pair.

## Reshaping by stacking and unstacking

![reshaping_stack](img/reshaping_stack.png)

Closely related to the `pivot()` method are the related `stack()` and `unstack()` methods available on `Series` and `DataFrame`. These methods are designed to work together with `MultiIndex` objects (see the section on `hierarchical indexing`). Here are essentially what these methods do:

* `stack`: “pivot” a level of the (possibly hierarchical) column labels, returning a `DataFrame` with an index with a new inner-most level of row labels.

* `unstack`: (inverse operation of `stack`) “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped `DataFrame` with a new inner-most level of column labels.

![reshaping_unstack](img/reshaping_unstack.png)

The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing section:

In [11]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)

index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])

df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])

df2 = df[:4]

df2

A         B
first second                    
bar   one    -0.372770 -2.128448
      two    -0.929253  1.711482
baz   one    -1.154158 -0.380045
      two    -0.046859 -0.418035

The `stack` function “compresses” a level in the `DataFrame`’s columns to produce either:

* A `Series`, in the case of a simple column Index.

* A `DataFrame`, in the case of a `MultiIndex` in the columns.

If the columns have a `MultiIndex`, you can choose which level to stack. The stacked level becomes the new lowest level in a `MultiIndex` on the columns:

In [12]:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -0.372770
               B   -2.128448
       two     A   -0.929253
               B    1.711482
baz    one     A   -1.154158
               B   -0.380045
       two     A   -0.046859
               B   -0.418035
dtype: float64

With a “stacked” `DataFrame` or `Series` (having a `MultiIndex` as the `index`), the inverse operation of `stack` is `unstack`, which by default unstacks the **last level**:

In [13]:
stacked.unstack()

A         B
first second                    
bar   one    -0.372770 -2.128448
      two    -0.929253  1.711482
baz   one    -1.154158 -0.380045
      two    -0.046859 -0.418035

In [14]:
stacked.unstack(1)

second        one       two
first                      
bar   A -0.372770 -0.929253
      B -2.128448  1.711482
baz   A -1.154158 -0.046859
      B -0.380045 -0.418035

In [15]:
stacked.unstack(0)

first          bar       baz
second                      
one    A -0.372770 -1.154158
       B -2.128448 -0.380045
two    A -0.929253 -0.046859
       B  1.711482 -0.418035

![reshaping_unstack_1](img/reshaping_unstack_1.png)

If the indexes have names, you can use the level names instead of specifying the level numbers:

In [17]:
stacked.unstack("second")

second        one       two
first                      
bar   A -0.372770 -0.929253
      B -2.128448  1.711482
baz   A -1.154158 -0.046859
      B -0.380045 -0.418035

![reshaping_unstack_0](img/reshaping_unstack_0.png)

Notice that the `stack` and `unstack` methods implicitly sort the index levels involved. Hence a call to `stack` and then `unstack`, or vice versa, will result in a sorted copy of the original `DataFrame` or `Series`:

In [18]:
index = pd.MultiIndex.from_product([[2, 1], ["a", "b"]])
df = pd.DataFrame(np.random.randn(4), index=index, columns=["A"])
df

A
2 a -0.094856
  b -0.928980
1 a -0.186210
  b  1.016235

In [19]:
all(df.unstack().stack() == df.sort_index())

True

The above code will raise a TypeError if the call to sort_index is removed.

### Multiple levels
You may also stack or unstack more than one level at a time by passing a list of levels, in which case the end result is as if each level in the list were processed individually.

In [20]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat", "long"),
        ("B", "cat", "long"),
        ("A", "dog", "short"),
        ("B", "dog", "short"),
    ],
    names=["exp", "animal", "hair_length"],
)

df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-0.104795,0.400806,2.459034,0.044143
1,0.554913,0.399143,0.645467,-0.756335
2,-0.435241,0.238767,1.401444,-1.707039
3,-1.007543,0.103140,-0.783322,0.630270


In [21]:
df.stack(level=["animal", "hair_length"])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.104795  0.400806
  dog    short        2.459034  0.044143
1 cat    long         0.554913  0.399143
  dog    short        0.645467 -0.756335
2 cat    long        -0.435241  0.238767
  dog    short        1.401444 -1.707039
3 cat    long        -1.007543  0.103140
  dog    short       -0.783322  0.630270

The list of levels can contain either level names or level numbers (but not a mixture of the two).

In [22]:
# df.stack(level=['animal', 'hair_length'])
# from above is equivalent to:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long        -0.104795  0.400806
  dog    short        2.459034  0.044143
1 cat    long         0.554913  0.399143
  dog    short        0.645467 -0.756335
2 cat    long        -0.435241  0.238767
  dog    short        1.401444 -1.707039
3 cat    long        -1.007543  0.103140
  dog    short       -0.783322  0.630270

### Missing data
These functions are intelligent about handling missing data and do not expect each subgroup within the hierarchical index to have the same set of labels. They also can handle the index being unsorted (but you can make it sorted by calling sort_index, of course). Here is a more complex example:



In [23]:
columns = pd.MultiIndex.from_tuples(
    [
        ("A", "cat"),
        ("B", "dog"),
        ("B", "cat"),
        ("A", "dog"),
    ],
    names=["exp", "animal"],
)

index = pd.MultiIndex.from_product(
    [("bar", "baz", "foo", "qux"), ("one", "two")], names=["first", "second"]
)

df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.233229  0.653944  1.835299  0.794578
      two     0.289322 -0.754708 -0.796277 -0.267863
baz   one    -1.599404 -0.140491  0.314109  0.142285
foo   one    -0.162875 -1.226721  0.611669  1.093423
      two     1.019976  0.189154 -0.276409 -0.553195
qux   two    -1.674473 -1.067543 -1.420049  0.456197

As mentioned above, stack can be called with a level argument to select which level in the columns to stack:

In [24]:
df2.stack("exp")

animal                 cat       dog
first second exp                    
bar   one    A   -0.233229  0.794578
             B    1.835299  0.653944
      two    A    0.289322 -0.267863
             B   -0.796277 -0.754708
baz   one    A   -1.599404  0.142285
             B    0.314109 -0.140491
foo   one    A   -0.162875  1.093423
             B    0.611669 -1.226721
      two    A    1.019976 -0.553195
             B   -0.276409  0.189154
qux   two    A   -1.674473  0.456197
             B   -1.420049 -1.067543

In [25]:
df2.stack("animal")

exp                         A         B
first second animal                    
bar   one    cat    -0.233229  1.835299
             dog     0.794578  0.653944
      two    cat     0.289322 -0.796277
             dog    -0.267863 -0.754708
baz   one    cat    -1.599404  0.314109
             dog     0.142285 -0.140491
foo   one    cat    -0.162875  0.611669
             dog     1.093423 -1.226721
      two    cat     1.019976 -0.276409
             dog    -0.553195  0.189154
qux   two    cat    -1.674473 -1.420049
             dog     0.456197 -1.067543

Unstacking can result in missing values if subgroups do not have the same set of labels. By default, missing values will be replaced with the default fill value for that data type, NaN for float, NaT for datetimelike, etc. For integer types, by default data will converted to float and missing values will be set to NaN.

In [26]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one     0.653944  1.835299
      two    -0.754708 -0.796277
foo   one    -1.226721  0.611669
qux   two    -1.067543 -1.420049

In [27]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar     0.653944 -0.754708  1.835299 -0.796277
foo    -1.226721       NaN  0.611669       NaN
qux          NaN -1.067543       NaN -1.420049

Alternatively, unstack takes an optional fill_value argument, for specifying the value of missing data.

In [28]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar     6.539436e-01 -7.547077e-01  1.835299e+00 -7.962773e-01
foo    -1.226721e+00 -1.000000e+09  6.116688e-01 -1.000000e+09
qux    -1.000000e+09 -1.067543e+00 -1.000000e+09 -1.420049e+00

### With a MultiIndex
Unstacking when the columns are a MultiIndex is also careful about doing the right thing:

In [29]:
df[:3].unstack(0)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
first        bar       baz       bar       baz       bar       baz       bar   
second                                                                         
one    -0.233229 -1.599404  0.653944 -0.140491  1.835299  0.314109  0.794578   
two     0.289322       NaN -0.754708       NaN -0.796277       NaN -0.267863   

exp               
animal            
first        baz  
second            
one     0.142285  
two          NaN

In [30]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar    -0.233229  0.289322  0.653944 -0.754708  1.835299 -0.796277  0.794578   
baz    -1.599404       NaN -0.140491       NaN  0.314109       NaN  0.142285   
foo    -0.162875  1.019976 -1.226721  0.189154  0.611669 -0.276409  1.093423   
qux          NaN -1.674473       NaN -1.067543       NaN -1.420049       NaN   

exp               
animal            
second       two  
first             
bar    -0.267863  
baz          NaN  
foo    -0.553195  
qux     0.456197

## Reshaping by melt

![reshaping_melt](img/reshaping_melt.png)

The top-level `melt()` function and the corresponding `DataFrame.melt()` are useful to massage a `DataFrame` into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the `var_name` and `value_name` parameters.

For instance,

In [31]:
cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    }
)
cheese

,first,last,height,weight
0,John,Doe,5.5,130
1,Mary,Bo,6.0,150


In [32]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [33]:
cheese.melt(id_vars=["first", "last"], var_name="quantity")

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


When transforming a DataFrame using `melt()`, the index will be ignored. The original index values can be kept around by setting the `ignore_index` parameter to `False` (default is `True`). This will however duplicate them.

*New in version 1.1.0*.

In [34]:
index = pd.MultiIndex.from_tuples([("person", "A"), ("person", "B")])

cheese = pd.DataFrame(
    {
        "first": ["John", "Mary"],
        "last": ["Doe", "Bo"],
        "height": [5.5, 6.0],
        "weight": [130, 150],
    },
    index=index,
)
cheese

first last  height  weight
person A  John  Doe     5.5     130
       B  Mary   Bo     6.0     150

In [35]:
cheese.melt(id_vars=["first", "last"])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [36]:
cheese.melt(id_vars=["first", "last"], ignore_index=False)

first last variable  value
person A  John  Doe   height    5.5
       B  Mary   Bo   height    6.0
       A  John  Doe   weight  130.0
       B  Mary   Bo   weight  150.0

Another way to transform is to use the `wide_to_long()` panel data convenience function. It is less flexible than `melt()`, but more user-friendly.

In [37]:
dft = pd.DataFrame(
    {
        "A1970": {0: "a", 1: "b", 2: "c"},
        "A1980": {0: "d", 1: "e", 2: "f"},
        "B1970": {0: 2.5, 1: 1.2, 2: 0.7},
        "B1980": {0: 3.2, 1: 1.3, 2: 0.1},
        "X": dict(zip(range(3), np.random.randn(3))),
    }
)
dft["id"] = dft.index
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,0.744658,0
1,b,e,1.2,1.3,0.812777,1
2,c,f,0.7,0.1,-0.251014,2


In [ ]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

## Combining with stats and GroupBy
It should be no shock that combining `pivot` / `stack` / `unstack` with GroupBy and the basic Series and DataFrame statistical functions can produce some very expressive and fast data manipulations.

In [38]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one    -0.233229  0.653944  1.835299  0.794578
      two     0.289322 -0.754708 -0.796277 -0.267863
baz   one    -1.599404 -0.140491  0.314109  0.142285
      two     1.105558  0.762259 -1.855329 -0.532554
foo   one    -0.162875 -1.226721  0.611669  1.093423
      two     1.019976  0.189154 -0.276409 -0.553195
qux   one    -1.010706  0.511416 -1.326121 -0.465026
      two    -1.674473 -1.067543 -1.420049  0.456197

In [39]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     0.801035  0.724261
      two    -0.253478 -0.511285
baz   one    -0.642647  0.000897
      two    -0.374886  0.114853
foo   one     0.224397 -0.066649
      two     0.371784 -0.182020
qux   one    -1.168413  0.023195
      two    -1.547261 -0.305673

In [40]:
# same result, another way
In [56]: df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one     0.801035  0.724261
      two    -0.253478 -0.511285
baz   one    -0.642647  0.000897
      two    -0.374886  0.114853
foo   one     0.224397 -0.066649
      two     0.371784 -0.182020
qux   one    -1.168413  0.023195
      two    -1.547261 -0.305673

In [41]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,-0.180119,0.154138
two,-0.019629,-0.652363


In [42]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,-0.283229,-0.364139
dog,0.083481,-0.134086


## Pivot tables
While `pivot()` provides general purpose pivoting with various data types (strings, numerics, etc.), pandas also provides `pivot_table()` for pivoting with aggregation of numeric data.

The function `pivot_table()` can be used to create spreadsheet-style pivot tables. See the `cookbook` for some advanced strategies.

It takes a number of arguments:

* `data`: a DataFrame object.

* `values`: a column or a list of columns to aggregate.

* `index`: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.

* `columns`: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.

* `aggfunc`: function to use for aggregation, defaulting to numpy.mean.

Consider a data set like this:



In [43]:
import datetime

df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 6,
        "B": ["A", "B", "C"] * 8,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 4,
        "D": np.random.randn(24),
        "E": np.random.randn(24),
        "F": [datetime.datetime(2013, i, 1) for i in range(1, 13)]
        + [datetime.datetime(2013, i, 15) for i in range(1, 13)],
    }
)
df

,A,B,C,D,E,F
0,one,A,foo,1.403130,-0.418091,2013-01-01
1,one,B,foo,0.886856,-0.245950,2013-02-01
2,two,C,foo,0.021251,1.266483,2013-03-01
3,three,A,bar,-0.100308,-0.550680,2013-04-01
4,one,B,bar,0.606261,1.863581,2013-05-01
5,one,C,bar,-0.463818,-0.321263,2013-06-01
6,two,A,foo,-1.257854,0.371389,2013-07-01
7,three,B,foo,-1.797417,0.323403,2013-08-01
8,one,C,foo,0.353625,0.450707,2013-09-01
9,one,A,bar,-0.454250,-1.127619,2013-10-01


We can produce pivot tables from this data very easily:

In [44]:
 pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])

C             bar       foo
A     B                    
one   A  0.130648 -0.136670
      B -0.039661 -0.164191
      C  0.418444  0.599156
three A -0.070974       NaN
      B       NaN -0.322757
      C -1.573322       NaN
two   A       NaN -0.420457
      B  0.697745       NaN
      C       NaN  0.844163

In [45]:
pd.pivot_table(df, values="D", index=["B"], columns=["A", "C"], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  0.261296 -0.273340 -0.141948       NaN       NaN -0.840914
B -0.079322 -0.328381       NaN -0.645515  1.395491       NaN
C  0.836887  1.198311 -3.146645       NaN       NaN  1.688327

In [46]:
pd.pivot_table(
    df, values=["D", "E"],
    index=["B"],
    columns=["A", "C"],
    aggfunc=np.sum,
)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  0.261296 -0.273340 -0.141948       NaN       NaN -0.840914 -2.648198   
B -0.079322 -0.328381       NaN -0.645515  1.395491       NaN  1.925883   
C  0.836887  1.198311 -3.146645       NaN       NaN  1.688327 -0.852977   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A -0.020641 -0.406997       NaN       NaN -2.655981  
B -0.890149       NaN -0.863851 -0.326851       NaN  
C -0.459452  0.599738       NaN       NaN  1.696062

The result object is a `DataFrame` having potentially hierarchical indexes on the rows and columns. If the `values` column name is not given, the pivot table will include all of the data that can be aggregated in an additional level of hierarchy in the columns:

In [47]:
pd.pivot_table(df, index=["A", "B"], columns=["C"])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.130648 -0.136670 -1.324099 -0.010320
      B -0.039661 -0.164191  0.962941 -0.445075
      C  0.418444  0.599156 -0.426489 -0.229726
three A -0.070974       NaN -0.203498       NaN
      B       NaN -0.322757       NaN -0.431926
      C -1.573322       NaN  0.299869       NaN
two   A       NaN -0.420457       NaN -1.327991
      B  0.697745       NaN -0.163425       NaN
      C       NaN  0.844163       NaN  0.848031

Also, you can use Grouper for index and columns keywords. For detail of Grouper, see Grouping with a Grouper specification.



In [48]:
pd.pivot_table(df, values="D", index=pd.Grouper(freq="M", key="F"), columns="C")

C,bar,foo
F,,
2013-01-31,NaN,-0.136670
2013-02-28,NaN,-0.164191
2013-03-31,NaN,0.844163
2013-04-30,-0.070974,NaN
2013-05-31,-0.039661,NaN
2013-06-30,0.418444,NaN
2013-07-31,NaN,-0.420457
2013-08-31,NaN,-0.322757
2013-09-30,NaN,0.599156


You can render a nice output of the table omitting the missing values by calling `to_string` if you wish:

In [49]:
table = pd.pivot_table(df, index=["A", "B"], columns=["C"])
print(table.to_string(na_rep=""))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.130648 -0.136670 -1.324099 -0.010320
      B -0.039661 -0.164191  0.962941 -0.445075
      C  0.418444  0.599156 -0.426489 -0.229726
three A -0.070974           -0.203498          
      B           -0.322757           -0.431926
      C -1.573322            0.299869          
two   A           -0.420457           -1.327991
      B  0.697745           -0.163425          
      C            0.844163            0.848031


Note that `pivot_table` is also available as an instance method on DataFrame,
i.e. `DataFrame.pivot_table()`.

### dding margins
If you pass `margins=True` to `pivot_table`, special `All` columns and rows will be added with partial group aggregates across the categories on the rows and columns:

In [50]:
df.pivot_table(index=["A", "B"], columns="C", margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.827171  2.177606  1.353716  0.277864  0.576675  0.843757
      B  0.913471  1.486405  1.009841  1.273697  0.281605  1.108169
      C  1.247706  0.347233  0.754983  0.148811  0.962278  0.573538
three A  0.041484       NaN  0.041484  0.490988       NaN  0.490988
      B       NaN  2.085483  2.085483       NaN  1.068195  1.068195
      C  1.084090       NaN  1.084090  0.476270       NaN  0.476270
two   A       NaN  1.184259  1.184259       NaN  2.403285  2.403285
      B  0.543120       NaN  0.543120  0.491061       NaN  0.491061
      C       NaN  1.163774  1.163774       NaN  0.591781  0.591781
All      0.991124  1.240281  1.077100  0.864873  1.111723  0.955578

## Cross tabulations
Use `crosstab()` to compute a cross-tabulation of two (or more) factors. By default `crosstab` computes a frequency table of the factors unless an array of values and an aggregation function are passed.

It takes a number of arguments

* `index`: array-like, values to group by in the rows.

* `columns`: array-like, values to group by in the columns.

* `values`: array-like, optional, array of values to aggregate according to the factors.

* `aggfunc`: function, optional, If no values array is passed, computes a frequency table.

* `rownames`: sequence, default `None`, must match number of row arrays passed.

* `colnames`: sequence, default `None`, if passed, must match number of column arrays passed.

* `margins`: boolean, default `False`, Add row/column margins (subtotals)

* `normalize`: boolean, {‘all’, ‘index’, ‘columns’}, or {0,1}, default `False`. Normalize by dividing all values by the sum of values.

Any `Series` passed will have their name attributes used unless row or column names for the cross-tabulation are specified

For example:

In [51]:
foo, bar, dull, shiny, one, two = "foo", "bar", "dull", "shiny", "one", "two"
a = np.array([foo, foo, bar, bar, foo, foo], dtype=object)
b = np.array([one, one, two, one, two, one], dtype=object)
c = np.array([dull, dull, shiny, dull, dull, shiny], dtype=object)
pd.crosstab(a, [b, c], rownames=["a"], colnames=["b", "c"])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     0    0     1
foo    2     1    1     0

If `crosstab` receives only two Series, it will provide a frequency table.

In [52]:
df = pd.DataFrame(
    {"A": [1, 2, 2, 2, 2], "B": [3, 3, 4, 4, 4], "C": [1, 1, np.nan, 1, 1]}
)
df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [53]:
pd.crosstab(df["A"], df["B"])

B,3,4
A,,
1,1,0
2,1,3


`crosstab` can also be implemented to Categorical data.

In [54]:
foo = pd.Categorical(["a", "b"], categories=["a", "b", "c"])
bar = pd.Categorical(["d", "e"], categories=["d", "e", "f"])
pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1


If you want to include all of data categories even if the actual data does not contain any instances of a particular category, you should set `dropna=False`.

For example:

In [55]:
pd.crosstab(foo, bar, dropna=False)

col_0,d,e,f
row_0,,,
a,1,0,0
b,0,1,0
c,0,0,0


### Normalization
Frequency tables can also be normalized to show percentages rather than counts using the `normalize` argument:

In [56]:
pd.crosstab(df["A"], df["B"], normalize=True)

B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


`normalize` can also normalize values within each row or within each column:

In [57]:
pd.crosstab(df["A"], df["B"], normalize="columns")

B,3,4
A,,
1,0.5,0.0
2,0.5,1.0


`crosstab` can also be passed a third `Series` and an aggregation function (`aggfunc`) that will be applied to the values of the third `Series` within each group defined by the first two `Series`:

In [58]:
pd.crosstab(df["A"], df["B"], values=df["C"], aggfunc=np.sum)

B,3,4
A,,
1,1.0,NaN
2,1.0,2.0


### Adding margins
Finally, one can also add margins or normalize this output.

In [ ]:
pd.crosstab(
   ....:     df["A"], df["B"], values=df["C"], aggfunc=np.sum, normalize=True, margins=True
   ....: )

## Tiling
The `cut()` function computes groupings for the values of the input array and is often used to transform continuous variables to discrete or categorical variables:

In [59]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])
pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60.0], (43.333, 60.0]]
Categories (3, interval[float64]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

If the `bins` keyword is an integer, then equal-width bins are formed. Alternatively we can specify custom bin-edges:

In [61]:
c = pd.cut(ages, bins=[0, 18, 35, 70])
c

[(0, 18], (0, 18], (0, 18], (0, 18], (18, 35], (18, 35], (18, 35], (35, 70], (35, 70]]
Categories (3, interval[int64]): [(0, 18] < (18, 35] < (35, 70]]

If the bins keyword is an IntervalIndex, then these will be used to bin the passed data.:

```python
pd.cut([25, 20, 50], bins=c.categories)
```

## Computing indicator / dummy variables
To convert a categorical variable into a “dummy” or “indicator” DataFrame, for example a column in a DataFrame (a Series) which has k distinct values, can derive a DataFrame containing k columns of 1s and 0s using get_dummies():

In [ ]:
df = pd.DataFrame({"key": list("bbacab"), "data1": range(6)})
pd.get_dummies(df["key"])

Sometimes it’s useful to prefix the column names, for example when merging the result with the original DataFrame:

In [ ]:
dummies = pd.get_dummies(df["key"], prefix="key")
dummies

In [ ]:
df[["data1"]].join(dummies)

This function is often used along with discretization functions like cut:

In [ ]:
values = np.random.randn(10)
values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

See also Series.str.get_dummies.

get_dummies() also accepts a DataFrame. By default all categorical variables (categorical in the statistical sense, those with object or categorical dtype) are encoded as dummy variables.

In [ ]:
df = pd.DataFrame({"A": ["a", "b", "a"], "B": ["c", "c", "b"], "C": [1, 2, 3]})
pd.get_dummies(df)

All non-object columns are included untouched in the output. You can control the columns that are encoded with the columns keyword.

In [ ]:
pd.get_dummies(df, columns=["A"])

Notice that the B column is still included in the output, it just hasn’t been encoded. You can drop B before calling get_dummies if you don’t want to include it in the output.

As with the Series version, you can pass values for the prefix and prefix_sep. By default the column name is used as the prefix, and ‘_’ as the prefix separator. You can specify prefix and prefix_sep in 3 ways:

* string: Use the same value for prefix or prefix_sep for each column to be encoded.

* list: Must be the same length as the number of columns being encoded.

* dict: Mapping column name to prefix.

In [ ]:
simple = pd.get_dummies(df, prefix="new_prefix")
simple

In [ ]:
from_list = pd.get_dummies(df, prefix=["from_A", "from_B"])
from_list

In [ ]:
from_dict = pd.get_dummies(df, prefix={"B": "from_B", "A": "from_A"})
from_dict

Sometimes it will be useful to only keep k-1 levels of a categorical variable to avoid collinearity when feeding the result to statistical models. You can switch to this mode by turn on drop_first.



In [ ]:
s = pd.Series(list("abcaa"))
pd.get_dummies(s)

In [ ]:
pd.get_dummies(s, drop_first=True)

When a column contains only one level, it will be omitted in the result.

In [ ]:
df = pd.DataFrame({"A": list("aaaaa"), "B": list("ababc")})
pd.get_dummies(df)

In [ ]:
pd.get_dummies(df, drop_first=True)

By default new columns will have np.uint8 dtype. To choose another dtype, use the dtype argument:

In [ ]:
df = pd.DataFrame({"A": list("abc"), "B": [1.1, 2.2, 3.3]})
pd.get_dummies(df, dtype=bool).dtypes

## Factorizing values
To encode 1-d values as an enumerated type use factorize():

In [ ]:
x = pd.Series(["A", "A", np.nan, "B", 3.14, np.inf])
x

In [ ]:
labels, uniques = pd.factorize(x)
labels

In [ ]:
uniques

Note that factorize is similar to numpy.unique, but differs in its handling of NaN:

>ote
>
>The following numpy.unique will fail under Python 3 with a TypeError because of an ordering bug. See also here.

In [ ]:
x = pd.Series(['A', 'A', np.nan, 'B', 3.14, np.inf])
pd.factorize(x, sort=True)

In [ ]:
np.unique(x, return_inverse=True)[::-1]

>Note
>
>If you just want to handle one column as a categorical variable (like R’s factor), you can use df["cat_col"] = pd.Categorical(df["col"]) or df["cat_col"] = df["col"].astype("category"). For full docs on Categorical, see the Categorical introduction and the API documentation.

## Examples
In this section, we will review frequently asked questions and examples. The column names and relevant column values are named to correspond with how this DataFrame will be pivoted in the answers below.

In [ ]:
np.random.seed([3, 1415])
n = 20
cols = np.array(["key", "row", "item", "col"])
df = cols + pd.DataFrame(
    (np.random.randint(5, size=(n, 4)) // [2, 1, 2, 1]).astype(str)
)

In [ ]:
df.columns = cols
df = df.join(pd.DataFrame(np.random.rand(n, 2).round(2)).add_prefix("val"))
df

### Pivoting with single aggregations
Suppose we wanted to pivot df such that the col values are columns, row values are the index, and the mean of val0 are the values? In particular, the resulting DataFrame should look like:

```
col   col0   col1   col2   col3  col4
row
row0  0.77  0.605    NaN  0.860  0.65
row2  0.13    NaN  0.395  0.500  0.25
row3   NaN  0.310    NaN  0.545   NaN
row4   NaN  0.100  0.395  0.760  0.24
```

This solution uses pivot_table(). Also note that aggfunc='mean' is the default. It is included here to be explicit.

In [ ]:
df.pivot_table(values="val0", index="row", columns="col", aggfunc="mean")

Note that we can also replace the missing values by using the fill_value parameter.

In [ ]:
df.pivot_table(
   .....:     values="val0",
   .....:     index="row",
   .....:     columns="col",
   .....:     aggfunc="mean",
   .....:     fill_value=0,
   .....: )

Also note that we can pass in other aggregation functions as well. For example, we can also pass in sum.

In [ ]:
df.pivot_table(
   .....:     values="val0",
   .....:     index="row",
   .....:     columns="col",
   .....:     aggfunc="sum",
   .....:     fill_value=0,
   .....: )

Another aggregation we can do is calculate the frequency in which the columns and rows occur together a.k.a. “cross tabulation”. To do this, we can pass size to the aggfunc parameter.

In [ ]:
df.pivot_table(index="row", columns="col", fill_value=0, aggfunc="size")

### Pivoting with multiple aggregations
We can also perform multiple aggregations. For example, to perform both a sum and mean, we can pass in a list to the aggfunc argument.

In [ ]:
df.pivot_table(
   .....:     values="val0",
   .....:     index="row",
   .....:     columns="col",
   .....:     a

Note to aggregate over multiple value columns, we can pass in a list to the values parameter.

In [ ]:
df.pivot_table(
   .....:     values=["val0", "val1"],
   .....:     index="row",
   .....:     columns="col",
   .....:     aggfunc=["mean"],
   .....: )

Note to subdivide over multiple columns we can pass in a list to the columns parameter.

In [ ]:
df.pivot_table(
   .....:     values=["val0"],
   .....:     index="row",
   .....:     columns=["item", "col"],
   .....:     aggfunc=["mean"],
   .....: )

## Exploding a list-like column
New in version 0.25.0.

Sometimes the values in a column are list-like.

In [ ]:
keys = ["panda1", "panda2", "panda3"]

In [136]: values = [["eats", "shoots"], ["shoots", "leaves"], ["eats", "leaves"]]

In [137]: df = pd.DataFrame({"keys": keys, "values": values})

In [138]: df

We can ‘explode’ the values column, transforming each list-like to a separate row, by using explode(). This will replicate the index values from the original row:

In [ ]:
df["values"].explode()

You can also explode the column in the DataFrame.

In [ ]:
df.explode("values")

Series.explode() will replace empty lists with np.nan and preserve scalar entries. The dtype of the resulting Series is always object.

In [ ]:
s = pd.Series([[1, 2, 3], "foo", [], ["a", "b"]])

In [142]: s

In [ ]:
s.explode()

Here is a typical usecase. You have comma separated strings in a column and want to expand this.

In [ ]:
df = pd.DataFrame([{"var1": "a,b,c", "var2": 1}, {"var1": "d,e,f", "var2": 2}])

In [145]: df

Creating a long form DataFrame is now straightforward using explode and chained operations

In [ ]:
df.assign(var1=df.var1.str.split(",")).explode("var1")